# Reconstruction with Truncations
_Author: Lucas Z. Brito_ 

This script performs reconstruction of the HS Hamiltonian including truncations 
of the operator basis.

In [1]:
using DataFrames
using CSV
using LinearAlgebra
using JSON
using Arpack
using JLD
using Plots

include("./src/hs_tools.jl")

ft_mat

In [21]:
function proj(v, mat)
	return mat * inv(mat' * mat) * mat' * v
end

function dec_mat(m, n_ops)
	shape = size(m)
	return m[1:(shape[1] - n_ops), 1:(shape[2] - n_ops)]
end

function dec_vec(v, n_ops)
	shape = size(m)
	return v[1:(shape[1] - n_ops)]
end

function decim_recon(mat, coeffs) 
	recons = []; n_ops = Int[]; cs = []; evals = []
	for i in 0:(size(mat)[1] - 1)
		mat_size = size(mat)[1] - i
		eig = eigen(dec_mat(mat, i))
		nullsp = eig.vectors[:, 1:min(2, mat_size)]
		current_evals = eig.values
		c = coeffs[1:end - i]
		evals = vcat(evals, current_evals)
		cs = vcat(cs, c)
		recons = vcat(recons, proj(c, nullsp))
		n_ops = vcat(n_ops, (mat_size) * ones(mat_size))
	end
	return Dict("recons" => recons, "n_ops" => n_ops, "coeffs" => cs, "evals" => evals)
end 


decim_recon (generic function with 1 method)

In [22]:
output_dir = "../data/hs_data/"

# Obtain WF, CM, FT CM 
n = 21
ft = ft_mat(trunc(Int, n/2))
cm_wf = load("../data/hs_data/cm_wf_n=$n.jld"); cm = cm_wf["cm"]; k = cm_wf["wf"]
ft_cm = ft * cm * ft'


# Obtain position space and FT HS coefficients
hs_analytical = hs_coeffs(n)
hs_ft = normalize(ft * hs_analytical)

pos_decim_recon = decim_recon(cm, hs_analytical)
recon = pos_decim_recon["recons"]
evals = pos_decim_recon["evals"]

ft_decim_recon = decim_recon(ft_cm, hs_ft)
recon_ft = ft_decim_recon["recons"]
n_ops = ft_decim_recon["n_ops"]
evals_ft_re, evals_ft_im = real(ft_decim_recon["evals"]), imag(ft_decim_recon["evals"])

([5.667840928772609e-17, 4.112077668407593e-15, 0.00013209696304354983, 0.0005502731341263993, 0.0013832833979162257, 0.0028589158059040788, 0.006623852113006995, 0.011851740930315687, 0.022045449322180378, 0.04928470613871323  …  0.0003280399699068628, 0.0018323306236391911, 0.006109539723911163, 0.0190698520187989, 0.0003433698361428474, 0.0018962374433623282, 0.006352061912811369, 0.0003548639055748747, 0.0019640254034673796, 0.0003671694469954053], [1.8060674101718382e-19, 4.684550000831782e-19, -3.0502506798466215e-19, 3.6614396951539454e-19, -8.386234343333078e-19, 9.863740825935858e-19, -1.8450268167371375e-18, 1.6027410424737292e-18, -8.633242160364914e-19, -2.876461232612652e-19  …  1.7083388142909645e-19, 4.00159709180584e-19, -9.788134465703702e-20, 1.471675400943232e-18, 1.5984356498443834e-19, -3.8162878483721277e-19, -1.1025169547091114e-19, 1.884371497359284e-19, -3.5784373918678833e-19, 2.1006417091906648e-19])

In [23]:
# Store reconstruction details in a dataframe.
recon_df = DataFrame(n_ops=n_ops, 
					 recon=recon, 
					 recon_ft_re=real(recon_ft), 
					 recon_ft_im=imag(recon_ft),
					 anal=pos_decim_recon["coeffs"],
					 anal_ft_re=real(ft_decim_recon["coeffs"]),
					 anal_ft_im=imag(ft_decim_recon["coeffs"]),
					 evals=evals, evals_ft_re=evals_ft_re, 
					 evals_ft_im=evals_ft_im)



Row,n_ops,recon,recon_ft_re,recon_ft_im,anal,anal_ft_re,anal_ft_im,evals,evals_ft_re,evals_ft_im
,Float64,Any,Float64,Float64,Any,Float64,Float64,Any,Float64,Float64
1,10.0,1.00749,0.345204,-0.0848419,1.00749,0.345204,-0.0848419,5.59716e-17,5.66784e-17,1.80607e-19
2,10.0,0.257596,0.289057,-0.0708532,0.257596,0.289057,-0.0708532,4.11216e-15,4.11208e-15,4.68455e-19
3,10.0,0.118882,0.261874,-0.0477631,0.118882,0.261874,-0.0477631,0.000132097,0.000132097,-3.05025e-19
4,10.0,0.0705262,0.248886,-0.0237849,0.0705262,0.248886,-0.0237849,0.000550273,0.000550273,3.66144e-19
5,10.0,0.0483752,0.244996,-5.24681e-16,0.0483752,0.244996,-8.72086e-18,0.00138328,0.00138328,-8.38623e-19
6,10.0,0.036613,0.248886,0.0237849,0.036613,0.248886,0.0237849,0.00285892,0.00285892,9.86374e-19
7,10.0,0.0298401,0.261874,0.0477631,0.0298401,0.261874,0.0477631,0.00662385,0.00662385,-1.84503e-18
8,10.0,0.0258273,0.289057,0.0708532,0.0258273,0.289057,0.0708532,0.0118517,0.0118517,1.60274e-18
9,10.0,0.0235459,0.345204,0.0848419,0.0235459,0.345204,0.0848419,0.0220454,0.0220454,-8.63324e-19


### Truncations

In [24]:
# Obtain eigenvalues for each decimation
evals = [eigen(dec_mat(cm, i)).values[1] for i in 0:(size(cm)[1] - 1)]
evals, evals_im = real(evals), imag(evals)
n_ops = [size(cm)[1] - i for i in 0:(size(cm)[1] - 1)]

evals_ft = [eigen(dec_mat(ft_cm, i)).values[1] for i in 0:(size(ft_cm)[1] - 1)]
evals_ft, evals_im_ft = real(evals), imag(evals)

df_evals = DataFrame(n_ops=n_ops, 
			   eval=evals,
			   eval_ft=evals_ft,
			   eval_im=evals_im,
			   eval_im_ft=evals_im_ft)

Row,n_ops,eval,eval_ft,eval_im,eval_im_ft
,Int64,Float64,Float64,Float64,Float64
1,10,5.59716e-17,5.59716e-17,0.0,0.0
2,9,3.95838e-15,3.95838e-15,0.0,0.0
3,8,2.07464e-9,2.07464e-9,0.0,0.0
4,7,1.07905e-8,1.07905e-8,0.0,0.0
5,6,3.10022e-8,3.10022e-8,0.0,0.0
6,5,1.07919e-7,1.07919e-7,0.0,0.0
7,4,2.69401e-7,2.69401e-7,0.0,0.0
8,3,1.31692e-6,1.31692e-6,0.0,0.0
9,2,4.14525e-6,4.14525e-6,0.0,0.0


In [25]:
df = recon_df # innerjoin(df_evals, recon_df, on=:n_ops)
fn = CSV.write(output_dir * "hs_truncations_n=$n.csv", df)
printstyled("Written decimation data to $fn\n", bold=true, color=:green)

Written decimation data to ../data/hs_data/hs_truncations_n=21.csv
